# Model Training and Evaluation


In [ ]:
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold

from snowflake.snowpark.context import get_active_session
session = get_active_session()

## Import Data From Database

In [ ]:
# Import Data
query =  """
SELECT *
FROM BROOKLYN_DATA.DATA.TRANSPORT_DATA;
"""
X = session.sql(query).to_pandas()
X = X.copy()
X

In [ ]:
avg_features = [col for col in X.columns if col.startswith('AVG_DIST')]

# Impute missing values with median
for col in avg_features:
    median_val = X[col].median()
    X[col] = X[col].fillna(median_val) 

In [ ]:
# Import Labels
query =  """
SELECT *
FROM BROOKLYN_DATA.DATA.DEMAND_SCORES_WEIGHT_LIFTED;
"""
y = session.sql(query).to_pandas()
# y = y.drop(columns=['SCORE_NORM_100', 'DEMAND_SCORE'])
y = y.values.ravel()
# y = y ** 0.5
y

## Create and Train Model

In [ ]:
pipe = make_pipeline(
    SimpleImputer(strategy='median'),
    HistGradientBoostingRegressor(random_state=42)
)

param_dist = {
    "histgradientboostingregressor__max_depth": [3, 4, 5, 6, 8],
    "histgradientboostingregressor__learning_rate": [0.01, 0.03, 0.05, 0.1],
    "histgradientboostingregressor__max_iter": [200, 300, 500],
    "histgradientboostingregressor__min_samples_leaf": [5, 10, 20],
}

kf = KFold(n_splits=5, shuffle=True, random_state=42)

search = RandomizedSearchCV(
    pipe,
    param_distributions=param_dist,
    n_iter=20,
    cv=kf,
    scoring="r2",
    random_state=42,
    n_jobs=-1
)

search.fit(X, y)

print(search.best_score_)
print(search.best_params_)

best_model = search.best_estimator_
rf_model = best_model

## Choose Best Model

In [ ]:
rf_model = search.best_estimator_
print(rf_model)

In [ ]:
train_preds = rf_model.predict(X)
train_r2 = r2_score(y, train_preds)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(rf_model, X, y, cv=kf, scoring='r2')
cv_r2 = cv_scores.mean()

print("Training R^2:", train_r2)
print("CV R^2:", cv_r2)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
import numpy as np

train_sizes, train_scores, cv_scores = learning_curve(
    rf_model,
    X,
    y,
    cv=5,
    scoring='r2',
    train_sizes=np.linspace(0.1, 1.0, 10),
    shuffle=True,
    random_state=42
)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
cv_mean = np.mean(cv_scores, axis=1)
cv_std = np.std(cv_scores, axis=1)

plt.figure(figsize=(12,8))
plt.plot(train_sizes, train_mean, 'o-', color='skyblue', label='Training R²')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color='skyblue', alpha=0.2)

plt.plot(train_sizes, cv_mean, 'o-', color='salmon', label='CV R²')
plt.fill_between(train_sizes, cv_mean - cv_std, cv_mean + cv_std, color='salmon', alpha=0.2)

plt.xlabel('Training Set Size')
plt.ylabel('R² Score')
plt.title('Learning Curve')
plt.ylim(0, 1)
plt.legend()
plt.grid(alpha=0.3)
plt.show()


## Import Data to Predict On

In [ ]:
# Import Data
query =  """
SELECT *
FROM BROOKLYN_DATA.DATA.TRANSPORT_DATA_5PCT;
"""
X_5pct = session.sql(query).to_pandas()
X_5pct = X_5pct.copy()

avg_features = [col for col in X_5pct.columns if col.startswith('AVG_DIST')]

# Impute missing values with median
for col in avg_features:
    median_val = X_5pct[col].median()
    X_5pct[col] = X_5pct[col].fillna(median_val) 

X_5pct = X_5pct.rename(columns={
    'TOTAL_POP_5PCT' : 'TOTAL_POP',
    'POP_16_OVER_5PCT' : 'POP_16_OVER',
    'COMMUTERS_BY_PUBLIC_TRANSPORTATION_5PCT' : 'COMMUTERS_BY_PUBLIC_TRANSPORTATION'
})
    
X_5pct

In [ ]:
# Import Data
query =  """
SELECT *
FROM BROOKLYN_DATA.DATA.TRANSPORT_DATA_10PCT;
"""
X_10pct = session.sql(query).to_pandas()
X_10pct = X_10pct.copy()

avg_features = [col for col in X_10pct.columns if col.startswith('AVG_DIST')]

# Impute missing values with median
for col in avg_features:
    median_val = X_10pct[col].median()
    X_10pct[col] = X_10pct[col].fillna(median_val) 
    
X_10pct = X_10pct.rename(columns={
    'TOTAL_POP_10PCT' : 'TOTAL_POP',
    'POP_16_OVER_10PCT' : 'POP_16_OVER',
    'COMMUTERS_BY_PUBLIC_TRANSPORTATION_10PCT' : 'COMMUTERS_BY_PUBLIC_TRANSPORTATION'
})
    
X_10pct

In [ ]:
# Import Data
query =  """
SELECT *
FROM BROOKLYN_DATA.DATA.TRANSPORT_DATA_15PCT;
"""
X_15pct = session.sql(query).to_pandas()
X_15pct = X_15pct.copy()

avg_features = [col for col in X_15pct.columns if col.startswith('AVG_DIST')]

# Impute missing values with median
for col in avg_features:
    median_val = X_15pct[col].median()
    X_15pct[col] = X_15pct[col].fillna(median_val) 

X_15pct = X_15pct.rename(columns={
    'TOTAL_POP_15PCT' : 'TOTAL_POP',
    'POP_16_OVER_15PCT' : 'POP_16_OVER',
    'COMMUTERS_BY_PUBLIC_TRANSPORTATION_15PCT' : 'COMMUTERS_BY_PUBLIC_TRANSPORTATION'
})
    
X_15pct

In [ ]:
# Import Data
query =  """
SELECT *
FROM BROOKLYN_DATA.DATA.TRANSPORT_DATA_20PCT;
"""
X_20pct = session.sql(query).to_pandas()
X_20pct = X_20pct.copy()

avg_features = [col for col in X_20pct.columns if col.startswith('AVG_DIST')]

# Impute missing values with median
for col in avg_features:
    median_val = X_20pct[col].median()
    X_20pct[col] = X_20pct[col].fillna(median_val) 

X_20pct = X_20pct.rename(columns={
    'TOTAL_POP_20PCT' : 'TOTAL_POP',
    'POP_16_OVER_20PCT' : 'POP_16_OVER',
    'COMMUTERS_BY_PUBLIC_TRANSPORTATION_20PCT' : 'COMMUTERS_BY_PUBLIC_TRANSPORTATION'
})
    
X_20pct

In [ ]:
# Import Data
query =  """
SELECT *
FROM BROOKLYN_DATA.DATA.TRANSPORT_DATA_25PCT;
"""
X_25pct = session.sql(query).to_pandas()
X_25pct = X_25pct.copy()

avg_features = [col for col in X_25pct.columns if col.startswith('AVG_DIST')]

# Impute missing values with median
for col in avg_features:
    median_val = X_25pct[col].median()
    X_25pct[col] = X_25pct[col].fillna(median_val) 

X_25pct = X_25pct.rename(columns={
    'TOTAL_POP_25PCT' : 'TOTAL_POP',
    'POP_16_OVER_25PCT' : 'POP_16_OVER',
    'COMMUTERS_BY_PUBLIC_TRANSPORTATION_25PCT' : 'COMMUTERS_BY_PUBLIC_TRANSPORTATION'
})
    
X_25pct

## Make Predictions

In [ ]:
y_5pct = rf_model.predict(X_5pct)
y_5pct

In [ ]:
y_10pct = rf_model.predict(X_10pct)
y_10pct

In [ ]:
y_15pct = rf_model.predict(X_15pct)
y_15pct

In [ ]:
y_20pct = rf_model.predict(X_20pct)
y_20pct

In [ ]:
y_25pct = rf_model.predict(X_25pct)
y_25pct